In [18]:
import numpy as np
from numpy.linalg import multi_dot
from numpy.linalg import qr
import numpy.linalg as npla
import scipy as sp
from scipy.linalg import block_diag, logm, eigvals
import pandas as pd
from multiprocessing import Pool
from functools import partial
import multiprocessing
from multiprocessing import set_start_method
from multiprocessing import get_context
import joblib
import time

In [19]:
thetacrit = np.pi/4

In [20]:
'''
These functions match to Holden's tlInCore and TlOutCore functions
They construct the A and B type transfer matricies
'''

def TAMatrix(theta, phi_one,phi_two,phi_three):
    '''mat = np.array(
    [[np.exp(1j*phi_two)*(1/np.cos(theta)),  -np.exp(-1j *phi_one)*np.tan(theta)],
     [np.exp(1j*(phi_two-phi_three))*(np.tan(theta)), np.exp(-1j*phi_one - 1j*phi_three)*(1/np.cos(theta))]]
    )'''
    matrix_one = np.array([[1,0],[0,np.exp(-1j*phi_three)]])
    matrix_two = np.array([[1,-np.sin(theta)],[-np.sin(theta),1]])
    matrix_three = np.array([[np.exp(1j*phi_two),0],[0,np.exp(-1j*phi_one)]])
    
    return (1/np.cos(theta))*multi_dot([matrix_one,matrix_two,matrix_three])


def TBMatrix(theta, phi_one,phi_two,phi_three):   
    matrix_one = np.array([[np.exp(-1j*phi_three),0],[0,np.exp(1j*phi_one)]])
    matrix_two = np.array([[1,np.cos(theta)],[np.cos(theta),1]])
    matrix_three = np.array([[-1,0],[0,np.exp(1j*phi_two)]])
    
    return (1/np.sin(theta)*multi_dot([matrix_one,matrix_two,matrix_three]))

In [21]:
'''
These functions here named TAS and TBS construct strips of TA type or TB type matrices with set strip_width
'''

#enter interger value for x that corresponds to sample width
def TAS(theta,strip_width):
    Tslist = [ 
        TAMatrix(theta,*(2*np.pi)*np.random.random_sample(3)) for i in range(strip_width)
    ]
    return block_diag(*Tslist)

def TBS(theta,strip_width):
    Tslist = [ 
        TBMatrix(theta,*(2*np.pi)*np.random.random_sample(3)) for i in range(strip_width-1)
    ]
    #extra = TBMatrix((2*np.pi)*np.random.random_sample(),*(2*np.pi)*np.random.random_sample(3))
    extra = TBMatrix(theta,*(2*np.pi)*np.random.random_sample(3))
    temp_mat = block_diag(extra[1,1],*Tslist,extra[0,0])    
    temp_mat[0,(2*strip_width)-1] = extra[1,0]
    temp_mat[(2*strip_width)-1,0] = extra[0,1]
    return temp_mat

'''
Since each TA strip is followed by a TB strip, instead of alternating functions, its easier to combine them into a single strip.
The final strip length that you run later is thus twice as long as specified. 
'''
def FullStrip(theta,strip_width):
    return np.matmul(TAS(theta,strip_width),TBS(theta,strip_width))

In [22]:
#runtime can be increased by tossing the R matrix after determining sum of logs of diagonals 
'''
Full transfer does the QR decomposition and calcuates the transfer matrix for the entire sample. 
Note that strip_length, as mentioned in the last cell, is actually the half strip length. 
The value calculated is actually double the specified length.
'''

def FullTransfer(strip_length,strip_width,theta):  
    Tone = FullStrip(theta,strip_width)
    qone,rone = qr(Tone)
    bigQ = qone
    rlog_one = np.log(np.absolute(rone.diagonal()))
    for i in range(strip_length-1):
        matrixb = np.matmul(FullStrip(theta,strip_width),bigQ)
        q,r = qr(matrixb)
        bigQ = q
        rlogs = np.log(np.absolute(r.diagonal()))
        rlog_one = np.add(rlogs,rlog_one)
    return rlog_one

We can get the eigenvalues of the final matrix because the elements (eigenvalues) of the final matrix are given by the sum of the logs of the diagonals for each rmatrix.

In [25]:
start_time = time.time()
vals = FullTransfer(10000,32,np.pi/4)
print('%s seconds' %(time.time() - start_time))

40.00912284851074 seconds


In [14]:
start_time = time.time()
theta_list = np.linspace(thetacrit,1.5,15)
partial_func = partial(FullTransfer,int(1e6),4)
final_array =[]
p = get_context("fork").Pool()
final_array.append(p.map(partial_func, theta_list))
final_array = np.concatenate(final_array[0]).ravel().tolist()
joblib.dump(final_array, '113_width8_length1e6.pkl')
print("--- %s minutes---" % ((time.time() - start_time)/60))

--- 73.53510979811351 minutes---


In [49]:
len(final_array)

104

In [ ]:
[thetacrit + i*np.pi/256 for i in range()]

In [46]:
np.linspace(thetacrit,3,13)

array([0.78539816, 0.96994832, 1.15449847, 1.33904862, 1.52359878,
       1.70814893, 1.89269908, 2.07724923, 2.26179939, 2.44634954,
       2.63089969, 2.81544985, 3.        ])

In [11]:
type(thetacrit)

float

In [9]:
np.log(0.5)

-0.6931471805599453